## Archivos NetCDF y xarray

**Temas Selectos de Modelación Numérica** <br>
Facultad de Ciencias, UNAM <br>
Semestre 2022-2

En este notebook aprenderemos qué son los archivos en formato netCDF con ayuda de la librería ``xarray`` y cómo manipular los datos. Esta breve introducción a netCDF y xarray es una versión traducida (y algo modificada) de los tres tutoriales de fundamentos de xarray de [Anderson Banihirwe](https://github.com/andersy005) que pueden encontrar aqui: https://github.com/xarray-contrib/xarray-tutorial/blob/master/online-tutorial-series. Hay un video (en inglés) en donde desarrollan este material: https://youtu.be/a339Q5F48UQ. Hay varios tutoriales de esta serie que les invito a explorar.


### Formato NetCDF
En Ciencias de la Tierra es muy común lidiar con estructuras de datos de muchas dimensiones (generalmente 3 espaciales + 1 temporal) como el estado del océano, la atmósfera, el interior de un planeta, etc. Es impráctico guardar estos datos en archivos de texto (como los .csv que hemos estado usando) porque necesitaríamos mucha capacidad de memoria para guardarlos, leerlos y procesarlos. Una de las mejores herramientas para manipular datos multidimensionales son los [netCDF](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#whatisit). Estos archivos almacenan datos en formato HDF (Hierarchical Data Format).

La estructura netCDF nos permite tener variables (los datos, por ejemplo temperaturas de un modelo), dimensiones (variables especiales que nos ayudan a dar estructura a los datos, como timepo, latitud, longitud, etc) y atributos (información acerca de las variables).

En esta sección aprenderemos a leer y escribir datos netCDF pero antes de eso necesitamos instalar el paquete `xarray` de python. Hay otras formas de leer y manipular archivos netCDF usando la biblioteca netcdf4, pero hoy aprenderemos como hacerlo usando `xarray`.

Ve a la terminal y escribe el siguente comando:

`conda install -c anaconda xarray `

Cuando haya terminado la instalación continúa con la sección.

### ¿Qué es xarray?

Los arreglos N-dimensionales de números son las estructuras de datos más comúnes en el cómputo científico (Ej: arreglos de numpy), pero carecen de una forma útil de asociarles metadatos (la información adicional acerca de los datos las coordenadas, cuándo se crearon, quién los creó, etc). Asociar metadatos y datos usualmente depende del individuo o del paquete de software específico que se use. Aquí entra `xarray`.

`xarray` expande las capacidades de los arreglos de NumPy y nos brinda un montón de facilidades para maipular datos.

La manera de interactuar con `xarray` esta basada en el modelo de datos de los archivos netCDF (variables, atributos y dimensiones).

`xarray` tiene dos estructuras fundamentales de datos:
    
   * `DataArray`, el "arreglo de datos" que contiene una sola variable multidimensional y a sus coordenadas,
   * `Dataset`, el "conjunto de datos" que contiene multiples variables que potencialemente comparten las mismas coordenadas


![data structures](images/xarray-data-structures.png)
Imagen de (https://github.com/xarray-contrib/xarray-tutorial/blob/master/online-tutorial-series/01_xarray_fundamentals.ipynb)


### ¿Cómo cargar datos NetCDF usando xarray?

In [ ]:
# importar xarray como xr. El nombre corto "xr" es convención pero puede ser el que quieras
import xarray as xr 

In [ ]:
# Carga el conjunto de datos de temperatura superficial del mar
ds = xr.open_dataset("data/sst.mnmean.nc", engine="netcdf4")

# representación HTML de xarray
ds

Explora el output (vista HTML) que te da xarray del conjunto de datos que cargamos en la variable `ds`. Por ejemplo, si das click en los íconos (hoja de papel y ¿discos?) al final de cada fila podrás ver información adicional de cada variable de datos y coordenada. ¿Qué variables contiene el conjunto de datos? ¿Cuáles son sus coordenadas? 

In [ ]:
# También puedes ver una representación modo texto. Para eso solo descomenta la línea siguiente
xr.set_options(display_style="text")

# Revisa la representación
ds.info()

#### Dataset (conjunto de datos)

`Dataset` es un contenedor de arreglos etiquetados (`DataArrays`) con dimensiones alineadas, similar a un diccionario de Python. Está diseñado como un representación en memoria de un conjunto de datos NetCDF.

Además de darnos acceso a los datos, como un diccionario de Python, los `Dataset`s tienen las siguentes propiedades:

|Atributo| Descripción|
|:--:|:--:|
|data_vars|`OrderedDict` de objetos `DataArray` correpondientes a variables de datos|
|dims| 	Mapeo de nombres de dimensiones a la longitud fija de cada dimensión (ej., {lat: 6, lon: 6, time: 8}).|
|coords| Un diccionario de arreglos (coordenadas) que etiquetan a cada punto (ej. arreglos 1D de números, objetos `datetime`|
|attrs |`OrderedDict` con metadatos arbitrarios del conjunto de datos.|

In [ ]:
# Variables de datos en el conjunto de datos (dataset)
ds.data_vars

In [ ]:
# selecciona la primera entrada a lo largo del primer eje (time) de la variable sst
ds.sst[0]

In [ ]:
# Grafica un paso de tiempo
ds.sst[0].plot()

In [ ]:
# dimensiones en el conjunto de datos
ds.dims

In [ ]:
# coordenadas en el conjunto de datos
ds.coords

In [ ]:
# atributos globales del conjunto de datos
ds.attrs

#### DataArray

El "arreglo de datos" `DataArray` es la implementación de un arreglo multidimensional con etiquetas. Algunas de sus propiedades clave son:

|Atributo| 	Descripción|
|:--:|:--:|
|data| 	arreglos numpy.ndarray o dask.array que contienen los datos.|
|dims| 	nombres de las dimensiones de cada eje. Ej:(x, y, z) (lat, lon, time).|
|coords| un contenedor estilo diccionario de arreglos (coordenadas) que etiquetan a cada punto(e.j., arreglos 1-D de números, objetos `datetime` o cadenas)|
|attrs| un `OrderedDict`que contiene metadatos y atributos (ej. unidades)|
|name |	nombre del arreglo|

In [ ]:
# Extrae la variable o dataArray sst
ds["sst"]  # Equivalente a ds.sst

In [ ]:
# El arreglo de datos (numpy) 
ds.sst.data

In [ ]:
# dimesniones del dataarray/variable 
ds.sst.dims

In [ ]:
# coordenadas del datarray/variable 
ds.sst.coords

In [ ]:
# atributos del dataarray/variable
ds.sst.attrs

#### Coordenadas vs. dimensiones
    
Los objetos `DataArray` dentro de un conjunto de datos `Dataset` pueden tener cualquier cantidad de dimensiones pero deben compartir el sistema de coordenadas.

Las coordenadas también pueden tener cualquier número de dimensiones pero denotan cantidades constantes e independientes, a diferencia de las cantidades que pertenecen a los datos.

La dimensión es solo un nombre que le damos a un eje, como tiempo.

In [ ]:
ds.dims

In [ ]:
ds.coords

In [ ]:
# extrae una variable de las coordenadas
ds.sst.lon

In [ ]:
# extrae una variable de las coordenadas de .coords
ds.coords["time"]

#### Atributos

Se pueden usar para guardar metadatos. ¿Cuáles? Pues depende de tu dominio y tus necesidades.

In [ ]:
# Ver atributos globales
ds.attrs

In [ ]:
# Ver atributos específicos de una variable (sst)
ds.sst.attrs

In [ ]:
# Guardar algunos atributos arbitrarios en una variable de datos/dataArray
ds.sst.attrs["mi_atributo"] = "Holaaa"
ds.sst.attrs

## Índices y selección de datos

Este material proviene del tutorial [02_indexing.ipynb](https://github.com/xarray-contrib/xarray-tutorial/blob/master/online-tutorial-series/02_indexing.ipynb) de este [repositorio](https://github.com/xarray-contrib/xarray-tutorial). 

En esta sección aprenderemos a seleccionar datos de acuerdo a su posición (`.isel`), a sus coordenadas (`.sel`), series de tiempo por tiempo/fecha y a buscar el valor más cecano a otro usando `.sel`.

¿Por qué necesitamos índices basados en etiquetas (digamos coordendas)?

Hay etiquetas inherentes a los datos geofísicos. Por ejemplo, una sere de tiempo incluye "estampas de tiempo" o pasos de tiempo que etiquetan a cada punto de la serie; los datos en espacio tienen coordenadas o etiquetas que nos dicen en qué punto fue tomada cada medición (ej. lat, lon, elevación).

### Índices posicionales en NumPy 

Cuando trabajamos con NumPy, indizamos de acuerdo a la posición (rebanadas/intervalos/enteros):

In [ ]:
t = ds["sst"].data  # toma los datos, arreglo de numpy 
t

In [ ]:
t.shape # revisemos el tamaño del arreglo

In [ ]:
# extraemos una serie de tiempo para una sola ubicación espacial
t[:, 10, 20]

Pero, ¿qué etiquetas le corresponden a 10 y 20? ¿Era lat/lon o lon/lat? ¿Dónde están las estampas de tiempo que van con esta serie de tiempo?

### Indices en xarray

xarray ofrece rutinas flexibles que combinan lo mejor de NumPy y [pandas](https://pandas.pydata.org/) para seleccionar datos:

In [ ]:
da = ds["sst"]  # Extrae arreglo de datos de la variable sst (DataArray)
da

* Los índices estilo NumPy aún funcionan en el Data Array, pero ahora se preservaron las etiquetas y metadatos:

In [ ]:
da[:, 20, 40]

Podemos seleccionar datos posicionalmente utilizando directamente el nombre de la dimensión: 

In [ ]:
# selecciona todo en la posición 60 (índice 60) a lo largo de la dimensión lat y la 
# posición 40 a lo largo de la dimensión lon y grafica
da.isel(lat=60, lon=40).plot() 

Podemos usar las etiquetas para indizar:

In [ ]:
da.sel(lat=-32, lon=80).plot()

In [ ]:
da.sel(lat=50.0, lon=200.0, time="2020")

In [ ]:
# Ejemplo de "slicing" (tomar una rebanada de los datos)

ds.sel(time=slice("2019-05", "2020-07"))

#### Encontrar el valor más cercano 

Podemos encontrar el dato más cercano a una posición (en tiepo o espacio) usando el método de vecinos más cercanos o "nearest neighbors":

In [ ]:
da.sel(lat=52.25, lon=251.8998, method="nearest")

En el ejemplo buscamos el valor de sst más cecrano a lat=52.25 grados y lon=251.8998 grados y el resultado fue el valor en lat=52.0 y lon=252.0 

**Todos los métodos que hemos visto también funcionan en `Dataset`s:**

In [ ]:
ds.sel(lat=52.25, lon=251.8998, method="nearest")

### Indices vectorizados

Como NumPy y Pandas, xarray nos permite indizar muchos elementos de una sola vez en forma vectorizada:

In [ ]:
# genera coordenadas para un transecto de puntos
lat_points = xr.DataArray([60, 80, 90], dims="points")
lon_points = xr.DataArray([250, 250, 250], dims="points")
lat_points

In [ ]:
lon_points

In [ ]:
# selecciona los puntos más cercanos a lo largo del transecto
da.sel(lat=lat_points, lon=lon_points, method="nearest").plot()

#### Indizar con where()

Podemos elegir un subconjunto de datos que cumple alguna condición con .where() (donde la condición es cierta, haz algo).

In [ ]:
# Reemplacemos los valores faltantes o NaN's (not a number) 
# con algún valor real arbitrario (-99)
ds.sst.where(ds.sst.notnull(), -99)

Ya no hay NaN's 

## Computar con los datos

Finalmente veremos cómo hacer aritmética básica con los datos y cómo agregar o reducir a lo largo de una dimensión.

Las operaciones aritméticas con un solo arreglo de datos (DataArray) se vectorizan automáticamente, como en NumPy:

In [ ]:
da = ds["sst"]
da

In [ ]:
da + 273.15 # tamibén es un vector

#### Métodos de agregación y reducción

Xarray soporta muchos de los métodos de agrecagión de NumPy (Ej: all, any, argmax, argmin, max, mean, median, min, prod, sum, std, var). A diferencia de NumPy, podemos usar los nombres de las dimensiones en vez de el escalar del eje:

[]

In [ ]:
da_mean = da.mean(dim="time")
da_mean

In [ ]:
da.std(dim=["lat", "lon"]).plot()

#### Broadcasting:

Broadcasting permite que un operador o función actúe en uno o más arreglos aún cuando no tienen la misma forma (shape) bajo ciertas condiciones. El siguiente esquema de Stephan Hoyer -- [xarray ECMWF Python workshop](https://docs.google.com/presentation/d/16CMY3g_OYr6fQplUZIDqVtG-SKZqsG8Ckwoj2oOqepU/edit#slide=id.g2b68f9254d_1_154) --  ilustra el resultado de operar en arreglos con distintas coordenadas:

![broadcasting](images/broadcasting.png)

In [ ]:
#Veamos los tamaños y coordenadas de del dataArray da
da.shape, da.dims

In [ ]:
# Veamos el tamaÑo y coordenadas después de tomar el promedio (dataArray da_mean)
da_mean.shape, da_mean.dims

In [ ]:
# Restemos el promedio (2D) al arreglo original (3D)
x = da - da_mean
x

Nos quedó algo de 3 dimensiones

### Operaciones de alto nivel: groupby, resample, rolling, coarsen, weighted

Xarray tiene otras funciones muy útiles de alto nivel que te permiten hacer operciones comunes:
   * `groupby` : Agrupa los datos en casillas o "bins" y reduce el tamaño
   * `resample` : Caso especial de `groupby`para el eje temporal. Permite cambiar la frecuencia de muestreo de los datos (upsample o downsample).
   * `rolling` : Opera en ventanas móviles. Ej. media móvil
   * `coarsen` : Disminuye la frecuencia de muestreo (Downsample)
   * `weighted` : Pondera los datos antes de aplicar alguna reducción.

#### groupby

In [ ]:
ds

In [ ]:
# Agrupa por estación del año
ds.groupby("time.season")

In [ ]:
# agrupa por día de la semana
ds.groupby("time.dayofweek")

In [ ]:
# calcula el promedio mensual
seasonal_mean = ds.groupby("time.season").mean()
seasonal_mean

In [ ]:
# Las estaciones no están en orden pero podemos acomodarlas en orden usando .reindex
seasonal_mean = seasonal_mean.reindex(season=["DJF", "MAM", "JJA", "SON"])
seasonal_mean

In [ ]:
seasonal_mean.sst.plot(col="season", robust=True, cmap="inferno")

#### resample

In [ ]:
# Cambia la frecuencia de muestreo a dos veces por mes
ds.sst.resample(time="2MS").mean()

#### rolling window operations

In [ ]:
# Média móvil con ventana tamaño 7
ds.sst.rolling(time=7).mean()

### Más recursos

Esto solo es una introducción a xarray y netcdf. Si quieren ir más allá les recomiendo estos notebooks y la documentación de xarray:
    
**Docs:**
* Estructuras de datos: http://xarray.pydata.org/en/latest/data-structures.html
* Lectura y escritura de archivos: https://xarray.pydata.org/en/stable/io.html
* Índices y selección de datos: https://xarray.pydata.org/en/stable/user-guide/indexing.html 

**Enlaces a notebooks**

Tutoriales de xarray:

* [Cómputo con xarray (extendida)](https://github.com/xarray-contrib/xarray-tutorial/blob/c133a80c2d911ef841ee6197f88ec0a0d87fbd94/scipy-tutorial/03_computation_with_xarray.ipynb)
* [Gráficas y visualización](https://github.com/xarray-contrib/xarray-tutorial/blob/c133a80c2d911ef841ee6197f88ec0a0d87fbd94/scipy-tutorial/04_plotting_and_visualization.ipynb)
